# Marker recognition performances
Marker recognition should be done in real time during rover exploration.\
Depending on the platform used to run the code, significant different time execution has been obtained. 

In [1]:
# Import libraries once
import numpy as np
import cv2
import cv2.aruco as aruco
import matplotlib.pyplot as plt

# Constant parameters used in Aruco methods
ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_50)

### Original, colored image
First we evaluate the plain function performance, given a non-filtered colored jpg image at **full resolution**. Photos are taken from smartphone with native resolution of 1960x4032:

In [2]:
# Read the image with the markers
queryImg = cv2.imread("test-imgs/two-A4-splitted_full.jpg")
print("Resolution: {}x{}".format(queryImg.shape[0], queryImg.shape[1]))

%timeit corners, ids, _ = aruco.detectMarkers(queryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)


Resolution: 1960x4032
227 ms ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Consideration about absolute timing cannot be provided, because it depends on the platform used.\
In Desktop PC the timeit returns an exection time of ?, contrariwise, on a SBC it took 227ms .

### Black/white image
We now try to do better. We consider a black/white image that is more light wrt memory occupation. We should also consider that camera image would be colored, so it is also relevant time spent for "cvtColor" function to convert the image. Image is full resolution also for this test.

In [3]:
grayQueryImg = cv2.cvtColor(queryImg, cv2.COLOR_BGR2GRAY)

%timeit corners, ids, _ = aruco.detectMarkers(grayQueryImg, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

182 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The only detectMarkers function seems to perform better with b/w image. The converted image can also be used later on the program to perform edge recognition, so must take notes about the speed up.

### Filtered image
We now try if filtering the image can speed up the search, due to the fact that less details are present in image. Also for this test, take full resolution image.

First try with a **bilateral filter**:

In [4]:
queryFiltered = cv2.bilateralFilter(queryImg, 9, 75, 75, cv2.BORDER_DEFAULT)

%timeit corners, ids, _ = aruco.detectMarkers(queryFiltered, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

156 ms ± 5.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


There is a speed up of about 31% wrt the detection without filtering, but we need to check if the filtering function is "time-friendly".

In [5]:
%timeit queryFiltered_second = cv2.bilateralFilter(queryImg, 9, 75, 75, cv2.BORDER_DEFAULT)

1.43 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So filtering is a very time-harvesting operation, one order of magnitude more than the plain search. If not needed for successive elaboration of the image, must consider carefully apply a filter.

We can try also with other filtering e.g. **Non-local Means Denoising**, already implemented in cv2:

In [10]:
%timeit queryFiltered = cv2.fastNlMeansDenoisingColored(queryImg, h=2, hColor=3, templateWindowSize=5, searchWindowSize=11)

10.8 s ± 270 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It turns out that despite of the good results, it's a very slow algorithm. Settings are also being lowered respect to the suggested, to get better results.

## Scaled image
We now try with a scaled version of the iamge. This could be not very good for edge detection, some light edge can be masked.

In [12]:
# Make a scaled version of the original image
scale_perc = 60 # percent of original size
width = int(queryImg.shape[1] * scale_perc / 100)
height = int(queryImg.shape[0] * scale_perc / 100)
dim = (width, height)

imgResized = cv2.resize(queryImg, dim, interpolation = cv2.INTER_AREA)

print("Resized Dimensions :", imgResized.shape)

Resized Dimensions : (1176, 2419, 3)


In [13]:
%timeit corners, ids, _ = aruco.detectMarkers(imgResized, ARUCO_DICT, parameters=ARUCO_PARAMETERS)

74 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


With a scaled version of the image we get a speedup of about 67%.

# Final considerations
Filtering the image not seems to be useful wrt the aruco recognition, regarding the computational time.\
Depending on the final camera specifications, the image can be upscaled or downscaled to get optimal performace.